# Simplified Ikeda sensitivity

In [ ]:
from jupyterthemes import jtplot
jtplot.style(theme='onedork', context='notebook', ticks=True, grid=False)

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
import numpy as np
import matplotlib.pyplot as plt
from pylab import rcParams
rcParams['figure.figsize'] = 15, 5

from rolldecayestimators.simplified_ikeda import calculate_roll_damping, SimplifiedIkedaInputError
from rolldecayestimators import equations
import sympy as sp
from rolldecayestimators import symbols
from rolldecayestimators.substitute_dynamic_symbols import lambdify
from mdldb.tables import Run
from rolldecayestimators.ikeda_estimator import IkedaEstimator
from rolldecayestimators.transformers import CutTransformer, LowpassFilterDerivatorTransformer, ScaleFactorTransformer, OffsetTransformer
from sklearn.pipeline import Pipeline

from rolldecay import database

import data
import copy
from rolldecay import database
from sympy.physics.vector.printing import vpprint, vlatex
from IPython.display import display, Math, Latex

In [ ]:
def calculate(row):
    
    LPP = row.lpp
    Beam = row.beam
    DRAFT = row.DRAFT
    
    PHI = row.phi_max
    lBK = row.BKL
    bBK = row.BKB
    OMEGA = row.omega0
    OG = (-row.kg+DRAFT)
    CB = row.CB
    CMID = row.A0
    
    s = pd.Series()
    try:
        B44HAT, BFHAT, BWHAT, BEHAT, BBKHAT = calculate_roll_damping(LPP,Beam,CB,CMID,OG,PHI,lBK,bBK,OMEGA,DRAFT)
    except SimplifiedIkedaInputError:
        return s
        
    s['B44HAT'] = B44HAT
    s['BFHAT'] = BFHAT
    s['BWHAT'] = BWHAT
    s['BEHAT'] = BEHAT
    s['BBKHAT'] = BBKHAT
    return s

In [ ]:
db = database.get_db()

In [ ]:
sql =  """
SELECT * from
loading_conditions
INNER JOIN models
    ON loading_conditions.model_number==models.model_number
    INNER JOIN ships
        ON ships.name == models.ship_name
"""

df = pd.read_sql_query(sql=sql, con=db.engine)

In [ ]:
mask = df['Volume']>0
df=df.loc[mask].copy()

In [ ]:
df.describe()

In [ ]:
df['DRAFT']=(df.TA + df.TF)/2
df['CB'] = df.Volume/(df.lpp*df.beam*df.DRAFT)
mask = df['CB'].abs()!=np.inf
df=df.loc[mask]

In [ ]:
df['CB'].hist(bins=30)

In [ ]:
mask = ((0 < df['CB']) & (df['CB'] <=1))
df = df.loc[mask]

In [ ]:
mask = df['Volume'] == df['Volume'].median()
ship = df.loc[mask].iloc[0]
ship['phi_max'] = 5
ship['omega0'] = 0.5

In [ ]:
ship.Volume/(ship.lpp*ship.beam*(ship.TA+ship.TF)/2)

In [ ]:
def variate_ship(ship,key,changes):
    N = len(changes)
    data = np.tile(ship.values,(N,1))
    
    df = pd.DataFrame(data=data, columns=ship.index)
    variations = changes*ship[key]
    df[key]=variations
    df.index = df[key].copy()
    
    return df
    

In [ ]:
N=30
changes = np.linspace(0.5,1.5,N)
df = variate_ship(ship=ship, key='lpp', changes=changes)

In [ ]:
result = df.apply(func=calculate, axis=1)

In [ ]:
result.plot()

In [ ]:
def plot_variation(ship=ship, key='lpp', changes=None, ax=None):

    if changes is None:
        N=30
        changes = np.linspace(0.5,1.5,N)
    
    if ax is None:
        fig,ax=plt.subplots()
    
    df = variate_ship(ship=ship, key=key, changes=changes)
    result = df.apply(func=calculate, axis=1)
    result['change factor']=changes
    result.plot(x='change factor',ax=ax)
    
    return ax
    

In [ ]:
plot_variation(ship=ship, key='lpp')

In [ ]:
variations = ['lpp','beam','DRAFT','phi_max','BKL','BKB','omega0','kg','CB','A0']
N=30
changes = np.linspace(0.7,1.32,N)

ylims=[]
axes=[]
for variation in variations:
    
    fig,ax=plt.subplots()
    plot_variation(ship=ship, key=variation, ax=ax, changes=changes)
    ax.set_title('Variation of %s: %0.2f' % (variation, ship[variation]))
    
    ylims.append(list(ax.get_ylim()))
    axes.append(ax)
    
for ax in axes:
    ax.set_ylim(np.min(ylims),np.max(ylims))
    